In [8]:
import molsysmt as msm
import pyunitwizard as puw

In [2]:
dialanine = msm.build.build_peptide('AceAlaNme')

In [4]:
msm.structure.center(dialanine, in_place=True)

In [5]:
dialanine = msm.build.solvate(dialanine, box_shape='rhombic dodecahedral',
                              clearance='14.0 angstroms')

In [6]:
msm.info(dialanine)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_structures
molsysmt.MolSys,649,212,210,2,210,2,209,1,1


In [7]:
topology = msm.convert(dialanine, to_form='openmm.topology')

In [9]:
positions = msm.get(dialanine, coordinates=True)
positions = puw.convert(positions[0], to_form='openmm.unit')